<a href="https://colab.research.google.com/github/nataliariano/PronosticoVentas/blob/main/PronosticoVentas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -Uq upgini catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.9/913.9 kB 58.6 MB/s eta 0:00

In [2]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19_000, random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [3]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

In [4]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [6]:
from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType
enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features, test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION



Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=b1408ace-f09f-4aed-b36d-2038fcc6b9d1
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


28 relevant feature(s) found with the search keys: ['date']


f_events_date_week_sin1_847b5db1,0.0318,100.0000,"-0.9749, -0.9749, -0.9749",Upgini,Calendar data,Free
f_events_date_year_cos1_9014a856,0.0304,100.0000,"0.8786, 0.2464, 0.5124",Upgini,Calendar data,Free
f_weather_date_weather_umap_48_b39cd0c4,0.0289,100.0000,"5.4207, 4.5063, 4.894",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_31_fa6d9a99,0.0273,100.0000,"4.6858, 5.0345, 4.9072",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_34_c3ef5b4f,0.0160,100.0000,"5.2741, 5.6683, 4.7807",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_47_5123ef0a,0.0159,100.0000,"5.9943, 6.0797, 5.4051",Upgini,Weather & climate normals data,Premium
f_events_date_week_cos3_7525fe31,0.0157,100.0000,"0.6235, 0.6235, 1.0",Upgini,Calendar data,Free
f_weather_date_weather_pca_0_d7e0a1fc,0.0123,100.0000,"26.6796, 12.606, 29.6873",Upgini,Weather & climate normals data,Premium
f_weather_date_weather_umap_33_89bb7578,0.0055,100.0000,"4.58, 4.6991, 4.9606",Upgini,Weather & climate normals data,Premium
f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,0.0026,100.0000,"0.9694, 0.8097, 0.9632",Upgini,Markets data,Free
f_financial_date_stoxx_043cbcd4,0.0021,100.0000,"386.13, 349.28, 304.56",Upgini,Markets data,Free


Upgini,Weather & climate normals data,0.1090,12
Upgini,Calendar data,0.0788,4
Upgini,Markets data,0.0069,4
Upgini,World economic indicators,0.0032,7
Upgini,AutoFE: feature from Markets data,0.0005,1


We detected 113 outliers in your sample.
Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 19000
After dropping target outliers size: 18887
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15148,49.9541,309.9905,191.7268,118.2637
Eval 1,3739,58.0011,509.2897,375.4910,133.7987


In [10]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features, train_target,
    eval_set = [(test_features, test_target)],
    estimator = model,
    scoring = "mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15148,49.9541,0.255035,0.158709,0.096326
1,Eval 1,3739,58.0011,0.269488,0.194272,0.075216


In [11]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_train_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

[=========================                                   ] 43% Enriching dataset...


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=a6d1a421-f6c0-4439-9317-f669d410b374
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


[============================================================] 100% Finished

...

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history


Column name,Status,Errors
date,All valid,-



Running search request, search_id=f743232d-57e0-4b36-ad0a-3070257e73f4
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_events_date_week_sin1_847b5db1,f_events_date_year_cos1_9014a856,f_weather_date_weather_umap_48_b39cd0c4,f_weather_date_weather_umap_31_fa6d9a99,f_weather_date_weather_umap_34_c3ef5b4f,f_weather_date_weather_umap_47_5123ef0a,f_events_date_week_cos3_7525fe31,...,f_economic_date_cbpol_pca_9_bde660b4,f_weather_date_weather_umap_26_5d9cba3d,f_economic_date_cbpol_pca_2_33d6e3fc,f_economic_date_cbpol_pca_1_31e5f62c,f_weather_date_weather_umap_5_88023931,f_economic_date_cbpol_umap_4_c5ce4e90,f_weather_date_weather_pca_1_9e4af931,f_economic_date_cpi_pca_0_af5ec5cf,f_economic_date_cci_pca_5_b433b3c8,f_weather_date_weather_pca_3_006924ed
15213,2017-01-01,1,44,-0.781831,0.98522,4.547627,4.721385,5.687399,5.971668,-0.900969,...,-0.366616,4.808236,1.898013,-4.251421,4.574104,5.408277,-11.300474,56.532993,-0.256691,3.032704
15214,2017-01-01,4,25,-0.781831,0.98522,4.547627,4.721385,5.687399,5.971668,-0.900969,...,-0.366616,4.808236,1.898013,-4.251421,4.574104,5.408277,-11.300474,56.532993,-0.256691,3.032704
15215,2017-01-01,3,15,-0.781831,0.98522,4.547627,4.721385,5.687399,5.971668,-0.900969,...,-0.366616,4.808236,1.898013,-4.251421,4.574104,5.408277,-11.300474,56.532993,-0.256691,3.032704
15216,2017-01-01,9,50,-0.781831,0.98522,4.547627,4.721385,5.687399,5.971668,-0.900969,...,-0.366616,4.808236,1.898013,-4.251421,4.574104,5.408277,-11.300474,56.532993,-0.256691,3.032704
15217,2017-01-01,4,37,-0.781831,0.98522,4.547627,4.721385,5.687399,5.971668,-0.900969,...,-0.366616,4.808236,1.898013,-4.251421,4.574104,5.408277,-11.300474,56.532993,-0.256691,3.032704


In [21]:
model.fit(train_features, train_target)
preds = model.predict(test_features)
eval_metric(test_target.values, preds, "SMAPE")

[37.65141857448004]